In [1]:
# Jupyter notebook to test docker python SDK
!which python

/home/vpraveen/.virtualenvs/tlt_launcher/bin/python


In [2]:
import os
import sys

import docker
import requests

docker_client = docker.from_env()
required_digest = "sha256:0043e8f4221be7229919c2d2b9ba7d14a4a3a156642476440147253bb74ab5a5"

In [3]:
# Working with containers.
container_list = docker_client.containers.list()
api_client = docker.APIClient(base_url="unix://var/run/docker.sock")
# container = container_list[0]
# inspection_content = api_client.inspect_container(container.attrs["Id"])


In [4]:
def check_if_image_exists(image_list):
    assert isinstance(image_list, list), (
        "image_list should be a list."
    )
    for image in image_list:
        image_inspection_content = api_client.inspect_image(image.attrs["Id"])
        if image_inspection_content["RepoDigests"]:
            image_digest = image_inspection_content["RepoDigests"][0].split("@")[1]
            if image_digest == required_digest:
                return True
    return False

In [5]:
# Working with images.
image_list = docker_client.images.list()
found = check_if_image_exists(image_list)
if not found:
    print("Pulling container")
    try:
        api_client.pull(repository="stg.nvcr.io/nvidia/tlt-streamanalytics", tag="v3.0.0ga-dev")
    except requests.exceptions.HTTPError:
        raise IOError("Docker not found.")
    except docker.errors.APIError:
        raise ValueError("Docker pull failed.")
    print("Container pull complete.")

In [ ]:
# Update the list of dockers after the pull and check if it exists.
image_list = docker_client.images.list()
found = check_if_image_exists(image_list)
print(found)

In [13]:
# Instantiate the downloaded container.
import subprocess
import sys
device_requests = docker.types.DeviceRequest(count=-1, capabilities=[["gpu"]])
volumes = {
    "/home/vpraveen": {
        "bind": "/workspace/vpraveen",
        "mode": "rw"
    },
    "/home/scratch.metropolis2/users/vpraveen/scratch_space/kitti-experiments": {
        "bind": "/workspace/tlt-experiments",
        "mode": "rw"
    }
}
experiment_spec_path = "/workspace/examples/detectnet_v2/specs/detectnet_v2_train_resnet18_kitti.txt"
results_dir = "/workspace/tlt-experiments/detectnet_v2/run_training_from_dockerpy"
command = "detectnet_v2 train -e {} -r {} -k tlt_encode --gpus 1".format(
    experiment_spec_path, results_dir
)
print("Instantiate the container.")
container = docker_client.containers.create(
    "stg.nvcr.io/nvidia/tlt-streamanalytics@{}".format(required_digest),
    device_requests=[device_requests],
    tty=True,
    detach=True,
    volumes=volumes
)
container.start()
docker_command = 'docker exec -it {} "{}"'.format(container.id, command)
print("The docker command: {}".format(docker_command))
if type(container) == docker.models.containers.Container:
    print("Running the container command.")
    # rc = subprocess.call(docker_command, shell=True, stdout=sys.stdout)
    # exit_code, output_log = container.exec_run(command, detach=False, stream=False)
    # for line in output_log:
    #     print(line)
    # print("Output_log: {}".format(output_log))

    print("Stopping container.")
    # container.stop()

Instantiate the container.
Running the container command.


UnsupportedOperation: fileno

In [33]:
# print(output_log.decode())
for line in output_log:
    print(line.decode())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x98 in position 0: invalid start byte

In [ ]:
import subprocess
import sys
# try:
#     subprocess.call(
#         "docker run -rm --detach stg.nvcr.io/nvidia/tlt-streamanalytics@{} /bin/bash".format(required_digest),
#         stdout=sys.stdout,
#         shell=True)
# except subprocess.CalledProcessError:
#     raise IOError("Couldn't run the docker.")
try:
    subprocess.call(
        "docker run -d --rm -t --gpus 2 stg.nvcr.io/nvidia/tlt-streamanalytics:v3.0.0ga-dev",
        shell=True      
    )
except subprocess.CalledProcessError:
    raise IOError("Couldn't run the docker.")
# ! docker run --rm --gpus all stg.nvcr.io/nvidia/tlt-streamanalytics:v3.0.0ga-dev 'detectnet_v2 train --help'
!docker ps

In [ ]:
!docker container inspect a1bffdbb216f

In [ ]:
api_client.inspect_container("a1bffdbb216f")

In [11]:
container = docker_client.containers.get("ef825950b653")
container.stop()

In [ ]:
!docker ps